<a href="https://colab.research.google.com/github/patrickphatnguyen/Image-Captioning-Attention/blob/master/Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

Re-implementing https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

# Import libraries

In [0]:
!pip install tensorflow-gpu==2.0.0-rc0
import os

import tensorflow as tf

from PIL import Image
import numpy as np
from tqdm import tqdm # For progress bar
import h5py
import pickle
import re
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

     |████████████████████████████████| 380.5MB 126kB/s 
     |████████████████████████████████| 4.3MB 31.6MB/s 
     |████████████████████████████████| 501kB 43.6MB/s 


In [0]:
data_path = os.path.abspath('.') + "/drive/My Drive/Image-Captioning/"
image_path = data_path +'val2014/'

image_urls_file = data_path + 'image_urls.txt'
captions_file = data_path + 'image_captions.txt'
features_path = data_path + 'extracted_features/'
name_of_file = data_path + 'coco14.h5'
name_of_dataset = 'coco14'

caption_json_file = data_path + 'annotations/captions_val2014.json'

NUM_SAMPLES = 33000 # Number of samples to take from the datasets


# Download captions & dataset

In [0]:
# Grab the captions file & extract
captions_zip = tf.keras.utils.get_file('captions.zip',
                                          cache_subdir=data_path,
                                          origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                          extract = True)
os.remove(os.path.join(data_path,"captions.zip"))

252878848/252872794 [==============================] - 3s 0us/step


In [0]:
# Grab the data
name_of_zip = 'coco2014.zip'

if not os.path.exists(data_path + name_of_zip):
  # Grab the train data & extract
  train_zip = tf.keras.utils.get_file(name_of_zip,
                                      cache_subdir=data_path,
                                      origin = 'http://images.cocodataset.org/zips/val2014.zip',
                                      extract = True)
  # Remove after extracted 
  os.remove(os.path.join(data_path,name_of_zip))

  
  PATH = os.path.dirname(data_path)+'/coco2014'
else:
  PATH = os.path.abspath('.')+'/coco2014/'

6645014528/6645013297 [==============================] - 110s 0us/step


# Load captions & Image URL

In [0]:
# Grab the annotation dict 
with open(caption_json_file ,'r') as f:
  captions_json = json.load(f)['annotations']


def load_captions_and_urls(captions_json):
  image_urls = []
  image_captions = []
  for line in captions_json:
    full_path = image_path + 'COCO_val2014_'+ "%012d.jpg" %(line['image_id'])
    caption = "<start> " + line['caption'] + " <end>"
    image_urls.append(full_path)
    image_captions.append(caption)
  return image_urls,image_captions

image_urls, image_captions = load_captions_and_urls(captions_json)
  

In [0]:
# Shuffle the image
image_urls, image_captions = shuffle(image_urls,image_captions,random_state = 1)
# Grab only a portion of the data 
image_urls, image_captions = image_urls[:NUM_SAMPLES],image_captions[:NUM_SAMPLES]

In [0]:
# Create a mapping function for tf.data.Dataset
def load_image(image_path):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image,channels = 3)
  image = tf.image.resize(image,(299,299))
  image = tf.keras.applications.inception_v3.preprocess_input(image)
  return image, image_path


In [0]:
# Grab a pretrained model to  
image_model = tf.keras.applications.InceptionV3(include_top = False, weights = 'imagenet')
das_input  = image_model.input
das_output = image_model.layers[-1].output
feature_extractor_model = tf.keras.Model(das_input,das_output)

87916544/87910968 [==============================] - 1s 0us/step


In [0]:
# Unique image
unique_image_urls = sorted(set(image_urls))

# Construct a dataset
image_dataset = tf.data.Dataset.from_tensor_slices(unique_image_urls)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img,path in tqdm(image_dataset):
  batch_features = feature_extractor_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())



1489it [05:13,  2.29it/s]


In [0]:
def compute_max_len_captions(caption_list):
  return max([len(caption) for caption in caption_list])

# Tokenizer

In [0]:
vocab_size = 5000
Tokenizer  = tf.keras.preprocessing.text.Tokenizer(num_words=5000,oov_token="<unk>",filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
Tokenizer.fit_on_texts(image_captions)
train_seqs = Tokenizer.texts_to_sequences(image_captions)
max_len_caption = compute_max_len_captions(train_seqs)
seqs_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs,padding="post")

In [0]:
len(Tokenizer.index_word)

8492

In [0]:
# Add Padding to vocab
Tokenizer.word_index["<pad>"] = 0
Tokenizer.index_word[0] = "<pad>"

# Train - test Splitting

In [0]:
image_urls_train, image_urls_test, seqs_train, seqs_test = train_test_split(image_urls,
                                                     seqs_vector,
                                                     test_size = 0.1, 
                                                     random_state = 0)
len(image_urls_train),len(seqs_train),len(image_urls_test),len(seqs_test)

(29700, 29700, 3300, 3300)

In [0]:
batch_size = 64
buffer_size = 1000
embedding_dim = 256
units = 512
vocab_size = len(Tokenizer.word_index) + 1

def map_func(image_url,cap_vec):
  image_tensor = np.load(image_url.decode('utf-8') + '.npy')
  return image_tensor,cap_vec

# Build the model!

In [0]:
vocab_size

8493

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((image_urls_train,seqs_train))
train_dataset = train_dataset.map(lambda image_url,cap_vec: \
                                  tf.numpy_function(map_func,
                                                    [image_url,cap_vec],
                                                    [tf.float32,tf.int32]),
                                 num_parallel_calls = tf.data.experimental.AUTOTUNE)

In [0]:
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=buffer_size)

In [0]:
class CNN_Encoder(tf.keras.Model):
    def __init__(self,embedding_dim):
      super().__init__()
      self.fc = tf.keras.layers.Dense(embedding_dim,activation='relu')
      
    def call(self,x):
      return self.fc(x)

In [0]:
class Attention(tf.keras.Model):
  def __init__(self,units):
    super().__init__()
    self.W1 = tf.keras.layers.Dense(units) # For features from CNN
    self.W2 = tf.keras.layers.Dense(units) # For time hidden layers
    self.Attention = tf.keras.layers.Dense(1) # To decide which node to pay attention  
  def call(self,features,hidden):
    # Input: features, hidden. Output: context_features, attention_
    # features.shape == (batch_size, 64, embedding_dim), this can be get from CNN encoder
    # hidden.shape ==  (batch_size, hidden_dim)
    
    # Reshape dimension to later broadcast with the feature tensors
    hidden_reshape = tf.expand_dims(hidden,axis = 1) # (batch_size,1,hidden_dim)
    
    score = tf.nn.tanh(self.W1(hidden_reshape)+self.W2(features)) # (batch_size, 64, units)
    
    # Calculate score respect to each pixel
    attention_score = self.Attention(score) # (batch_size, 64, 1)
    
    # Scaling to 0->1 and amplify the weights using softmax  
    attention_weights = tf.nn.softmax(attention_score,axis=1) # (batch_size, 64, 1)
    
    # Apply attention weights to features
    context_features = attention_weights * features # (batch_size, 64, embedding_dim)
    
    # Reduce sum along axis 1 to reveal the context embedding picture wise
    context_features = tf.reduce_sum(context_features,axis=1) # (batch_size, embedding_dim)
 
    return context_features, attention_weights
  


In [0]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super().__init__()
    
    # Attributes
    self.embedding_dim = embedding_dim
    self.units = units
    self.vocab_size = vocab_size
    
    # Layers
    self.embedding = tf.keras.layers.Embedding(vocab_size,embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                  return_sequences = True,
                                  return_state = True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units, activation = 'relu')
    self.fc2 = tf.keras.layers.Dense(vocab_size)
    self.attention = Attention(self.units)
    
  def call(self, x, features, hidden):
    # defining attention as a seperate model
    context_features, attention_weights = self.attention(features, hidden)
 
    x = self.embedding(x)
  
    # Concat feature and hidden together to put in to GRU 

    # also, expand dimensions for time steps
    x = tf.concat([tf.expand_dims(context_features,1),x],axis=-1) 
    ### (batch_size, 1, embedding + hidden_size)
    
    # put through gru
    output, state = self.gru(x) 
    
    # pass through fc1
    x = self.fc1(output) # (batch_size, 1, units)
    
    
    # reshape 
    x = tf.reshape(output,(-1,x.shape[2])) #(batch_size, units)
    
    # pass through fc2
    x = self.fc2(x) # (batch_size, vocab_size)
    
    
    return x, state, attention_weights
  
  def test_step(self,x, features, hidden):
    
    # Get output and hidden from decoder
    x, hidden, _  = self(x, features, hidden)

    # Update new input for next decoder time steps (use ground truth to force)
    x = tf.argmax(x,1) # (batch_size,1)
    return x, hidden

  def test(self, image_tensor):
    num_samples = image_tensor.shape[0]
    
    # Put through CNN encoder
    features = encoder(image_tensor) 

    # Init hidden state to decoder
    hidden = self.init_state(num_samples) 

    # Init <start> tensor for input
    x = self.init_input(Tokenizer, num_samples)
    
    # Change type to concatenate
    result = tf.cast(self.init_input(Tokenizer, num_samples),dtype=tf.int64)

    for i in range(target.shape[1]):
      x, hidden = self.test_step(x, features, hidden)
      x = tf.expand_dims(x,1)
      result = tf.concat([result,x],1)
    return result
    
  def init_state(self,batch_size):
    return tf.zeros((batch_size,self.units))
  
  def init_input(self,Tokenizer,batch_size):
    return tf.expand_dims([Tokenizer.word_index['<start>']]*batch_size,1)
    

In [0]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction='none')

# Loss Function

In [0]:
def loss_func(y_true,y_pred):
  mask = tf.math.logical_not(tf.equal(y_true,0)) 
  # False if <pad>, otherwise True
  loss = loss_obj(y_true,y_pred)
  mask = tf.cast(mask, dtype = loss.dtype)
  loss = loss * mask # Only calculate loss if not <pad> position
  return tf.reduce_mean(loss)

In [0]:
@tf.function
def train_iter(image_tensor, target):
  batch_size = image_tensor.shape[0]
  
  # Init loss
  loss = 0
  
  # Put through CNN encoder
  features = encoder(image_tensor) 
  
  # Init hidden state to decoder
  hidden = decoder.init_state(batch_size) 
  
  # Init <start> tensor for input
  x = decoder.init_input(Tokenizer, batch_size)
  
  with tf.GradientTape() as t:
    for i in range(target.shape[1]):
      # Get output and hidden from decoder
      x, hidden, _  = decoder(x, features, hidden)
      
      # Compute loss function
      loss += loss_func(target[:,i], x)
      
      # Update new input for next decoder time steps (use ground truth to force)
      x = tf.expand_dims(target[:,i],1)
      

  total_loss = loss / image_tensor.shape[0]
  
  train_params =  encoder.trainable_variables + decoder.trainable_variables 
  
  gradients = t.gradient(loss,train_params) # calculate gradient respected to param
  
  optimizer.apply_gradients(zip(gradients,train_params))
  
  return loss, total_loss
  

In [0]:
epochs = 5 
for i in range(epochs):
  print('EPOCHS: ',i)
  total_loss = None
  for (batch,(img_tensor,target)) in enumerate(train_dataset):
    
    loss,total_loss = train_iter(img_tensor, target)
    if batch % 100 == 0:
      print('batch ' + str(batch) + ' loss: ',total_loss.numpy())
  print('total loss: ',total_loss)

  

EPOCHS:  0
batch 0 loss:  1.8006495
batch 100 loss:  0.8624797
batch 200 loss:  0.7513491
batch 300 loss:  0.7459854
batch 400 loss:  0.6877638
total loss:  tf.Tensor(9.457412, shape=(), dtype=float32)
EPOCHS:  1
batch 0 loss:  0.6242118
batch 100 loss:  0.633465
batch 200 loss:  0.6228994
batch 300 loss:  0.5797832
batch 400 loss:  0.64614356
total loss:  tf.Tensor(10.482074, shape=(), dtype=float32)
EPOCHS:  2
batch 0 loss:  0.563726
batch 100 loss:  0.5992195
batch 200 loss:  0.52837044
batch 300 loss:  0.5725616


In [271]:
result = None
for image_tensor,_ in train_dataset:
  result = decoder.test(image_tensor[0:1])
Tokenizer.sequences_to_texts(result.numpy())

KeyboardInterrupt: ignored

In [272]:
Tokenizer.sequences_to_texts(result.numpy())

['<start> <start> a man is a man is a man is a man is a man is a man is a man is a man is a man is a man is a man is a man is a man is a man is a man is a man is']